In [1]:
import pandas as pd

In [2]:
tickers = pd.read_csv('ticker_list_2.csv')
tickernames = tickers['name']
biotech_tickers = tickernames.to_list()

In [3]:
import random
import string

def random_string(length):
    x = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(length))
    return x

random_string(13)

'ujyuthpou03xl'

In [4]:
import requests

In [5]:
for ticker in biotech_tickers:
    query = {'s': random_string(13), 't': ticker}
    res = requests.get('https://www.macrotrends.net/assets/php/stock_data_download.php', params=query)
    decoded = res.content.decode('utf-8')
    with open(f'tickers/{ticker}.csv', 'w') as f:
        f.write(decoded)
        f.close()

## Getting data for the same time period

At this point, we have CSV files for each stock saved as ticker.csv in the `tickers` folder in this directory.

### Remove the download message

The real data starts at the CSV column headings, with `date`, so we can remove everything before that

In [7]:
for ticker in biotech_tickers:
    file_path = f'tickers/{ticker}.csv'
    with open(file_path, 'r+') as f:
        content = f.read()
        date_index = content.index('date')
        cropped = content[date_index:]
        f.seek(0)
        f.write(cropped)
        f.close()

## Reduce data to just the opening prices

In [8]:
for ticker in biotech_tickers:
    file_path = f'tickers/{ticker}.csv'
    df = pd.read_csv(file_path)
    try:
        df = df.drop(['high', 'low', 'close', 'volume'], axis=1)
        df.to_csv(file_path)
    except Exception as e:
        print(f'Error for {ticker}: {e}')

## Reducing data to a timeframe

In [9]:
for year in range(2013, 2023, 1):
    count = 0
    for ticker in biotech_tickers:
        file_path = f'tickers/{ticker}.csv'
        df = pd.read_csv(file_path)
        earliest_date = df['date'][0]
        earliest_date_year = int(earliest_date[:4])
        if earliest_date_year <= year:
            count += 1

    print(f'{count} stocks have data starting in a year before {year}')

84 stocks have data starting in a year before 2013
85 stocks have data starting in a year before 2014
86 stocks have data starting in a year before 2015
87 stocks have data starting in a year before 2016
90 stocks have data starting in a year before 2017
92 stocks have data starting in a year before 2018
97 stocks have data starting in a year before 2019
97 stocks have data starting in a year before 2020
99 stocks have data starting in a year before 2021
100 stocks have data starting in a year before 2022


This year, we're going with 2015 onwards

In [10]:
year = 2015

In [12]:
tickers_2015 = []

for ticker in biotech_tickers:
    df = pd.read_csv(f'tickers/{ticker}.csv')
    earliest_date = df['date'][0]
    earliest_date_year = int(earliest_date[:4])
    if earliest_date_year <= year:
        df.to_csv(f'tickers_2015/{ticker}.csv')
        tickers_2015.append(ticker)

Now, we need to remove values before 1 Jan 2015

In [13]:
def date_str_valid(date_str):
    try:
        year = int(str(date_str)[:4])
    except:
        return False
    if year >= 2015:
        return True
    return False    

In [14]:
df = df[df.apply(lambda x: date_str_valid(x['date']), axis = 1)]
df

,Unnamed: 0,date,open
4826,4826,2015-01-02,75.84
4827,4827,2015-01-05,75.36
4828,4828,2015-01-06,74.89
4829,4829,2015-01-07,73.87
4830,4830,2015-01-08,74.87
...,...,...,...
6778,6778,2022-09-08,90.01
6779,6779,2022-09-09,91.80
6780,6780,2022-09-12,93.23
6781,6781,2022-09-13,94.00


In [15]:
for ticker in tickers_2015:
    original_file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(original_file_path)
    df = df.dropna()
    df = df[df.apply(lambda x: date_str_valid(x['date']), axis = 1)]
    df.to_csv(original_file_path)

Now we need to remove data from 2020 or later

In [16]:
def date_str_before_2020(date_str):
    try:
        year = int(str(date_str)[:4])
    except:
        return False
    if year < 2020:
        return True
    return False    

In [17]:
for ticker in tickers_2015:
    original_file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(original_file_path)
    df = df.dropna()
    df = df[df.apply(lambda x: date_str_before_2020(x['date']), axis = 1)]
    df.to_csv(original_file_path)

We now have a bunch of junk which we should remove from each of the files

In [18]:
for ticker in tickers_2015:
    file_path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(file_path)
    df = df[['date', 'open']]
    df.to_csv(file_path)

## Preparing Data

Now we need to merge these all into one dataframe / CSV file. This should have the date as the column and opening price for that stock as the row

First, we need to change the files so that each have them have the date as the column and opening price as the value

In [19]:
for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    df = df[['date', 'open']]
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.drop(['date'])
    df.insert(loc=0, column='', value=ticker)
    df.to_csv(path, index=False)

Now, we need to merge all the dataframes into one mega DF

In [20]:
dfs = []

for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    dfs.append(df)

big_boi = pd.concat(dfs, axis=0)
big_boi

,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,UNH,90.2065,89.1400,88.7223,87.9491,90.2687,92.8016,92.2772,92.3750,90.3042,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
0,JNJ,85.9042,85.4381,85.3236,84.9720,86.7301,87.0899,86.0023,86.2231,84.9311,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
0,LLY,59.1298,59.0959,59.2569,59.3501,59.3247,59.3501,59.5112,59.3840,58.2823,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
0,PFE,22.3480,22.4052,22.3480,22.6696,23.1127,23.2128,23.3486,23.4987,23.0198,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
0,ABBV,47.2114,47.1251,47.2114,46.4560,49.0389,47.9813,47.7294,47.6489,45.4013,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,JAZZ,165.9600,162.2000,160.5300,160.6100,164.9500,163.4700,162.0300,161.7300,160.3100,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900
0,SRPT,14.5800,14.7500,15.0300,14.4900,15.0800,14.5600,13.4100,12.0100,11.5800,...,135.7900,130.6800,126.1900,126.3500,136.9900,135.7100,133.3900,133.0700,129.4500,127.6200
0,NVCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.2500,82.6500,81.8200,85.3500,87.3400,88.7500,89.6400,87.5500,87.4700,84.9300
0,RDY,47.8324,46.9533,45.7811,45.4975,46.1498,47.8135,48.5981,48.9195,48.4847,...,39.2188,39.8740,39.7567,39.6002,40.0990,39.8251,39.6980,39.9719,39.7273,39.5904


In [21]:
big_boi.reset_index()
big_boi

,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,UNH,90.2065,89.1400,88.7223,87.9491,90.2687,92.8016,92.2772,92.3750,90.3042,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
0,JNJ,85.9042,85.4381,85.3236,84.9720,86.7301,87.0899,86.0023,86.2231,84.9311,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
0,LLY,59.1298,59.0959,59.2569,59.3501,59.3247,59.3501,59.5112,59.3840,58.2823,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
0,PFE,22.3480,22.4052,22.3480,22.6696,23.1127,23.2128,23.3486,23.4987,23.0198,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
0,ABBV,47.2114,47.1251,47.2114,46.4560,49.0389,47.9813,47.7294,47.6489,45.4013,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,JAZZ,165.9600,162.2000,160.5300,160.6100,164.9500,163.4700,162.0300,161.7300,160.3100,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900
0,SRPT,14.5800,14.7500,15.0300,14.4900,15.0800,14.5600,13.4100,12.0100,11.5800,...,135.7900,130.6800,126.1900,126.3500,136.9900,135.7100,133.3900,133.0700,129.4500,127.6200
0,NVCR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.2500,82.6500,81.8200,85.3500,87.3400,88.7500,89.6400,87.5500,87.4700,84.9300
0,RDY,47.8324,46.9533,45.7811,45.4975,46.1498,47.8135,48.5981,48.9195,48.4847,...,39.2188,39.8740,39.7567,39.6002,40.0990,39.8251,39.6980,39.9719,39.7273,39.5904


Seems like some rows are missing in `big_boi`

In [22]:
import numpy as np
columns_all = np.array([column for column in [columns for columns in [df.columns for df in dfs]]])
first_dates = [i[1] for i in columns_all]
(first_dates.sort())
first_dates[-1]

/var/folders/8m/xxtd2bpx07b2133k5m87vfq80000gn/T/ipykernel_3472/1148072085.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  columns_all = np.array([column for column in [columns for columns in [df.columns for df in dfs]]])


'2015-10-02'

The latest first date is  '2015-12-22'. This means we will have to reduce all of our data so that it starts on the 22nd of December 2015.

In [23]:
dates_to_remove = []
for i in range(1, 12, 1):
    month_code = ''
    if i <= 9:
        month_code = f'0{i}'
    else:
        month_code = str(i)
    for k in range(1, 32, 1):
        if k <= 9:
            date = f'2015-{month_code}-0{k}'
        else:
            date = f'2015-{month_code}-{k}'
        dates_to_remove.append(date)

for j in range(1, 22, 1):
    if j <= 9:
        date = f'2015-12-0{j}'
    else:
        date = f'2015-12-{j}'
    dates_to_remove.append(date)

In [24]:
'2015-10-06' in dates_to_remove

True

In [25]:
dfs = []

for ticker in tickers_2015:
    path = f'tickers_2015/{ticker}.csv'
    df = pd.read_csv(path)
    df = df.drop(dates_to_remove, axis=1, errors='ignore')
    dfs.append(df)

big_boi = pd.concat(dfs, axis=0)
big_boi

,Unnamed: 0,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,UNH,106.7421,106.4802,107.0491,106.8053,107.3562,108.4038,107.1485,105.5860,105.4145,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
0,JNJ,85.8198,86.8303,86.8387,86.8893,87.0240,87.6724,87.0493,85.6514,84.6998,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
0,LLY,75.0504,75.1287,74.7981,74.4501,74.6763,75.2070,74.5719,72.5537,72.1187,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
0,PFE,24.0945,24.0650,24.0059,24.0798,24.0576,24.1832,23.9837,23.5258,23.7695,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
0,ABBV,42.0324,43.0286,43.4003,43.3185,43.8761,44.1586,44.2329,43.1624,43.1104,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,JAZZ,139.9900,144.1500,144.2400,141.7800,142.6500,142.5600,142.5500,137.8300,138.3500,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900
0,SRPT,37.9200,37.4800,38.3500,38.3100,37.8100,39.5000,38.5500,37.8500,38.2000,...,135.7900,130.6800,126.1900,126.3500,136.9900,135.7100,133.3900,133.0700,129.4500,127.6200
0,NVCR,27.5000,27.1200,26.3300,25.9100,24.9600,24.0100,22.8700,22.0000,22.7900,...,82.2500,82.6500,81.8200,85.3500,87.3400,88.7500,89.6400,87.5500,87.4700,84.9300
0,RDY,43.0524,43.2615,42.8908,44.3259,44.0123,44.3544,44.1548,43.4705,43.5466,...,39.2188,39.8740,39.7567,39.6002,40.0990,39.8251,39.6980,39.9719,39.7273,39.5904


Now for some cleaning

In [26]:
big_boi = big_boi.reset_index(drop=True)

In [27]:
big_boi.rename(columns={'Unnamed: 0': 'ticker'}, inplace=True)
big_boi.head()

,ticker,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,UNH,106.7421,106.4802,107.0491,106.8053,107.3562,108.4038,107.1485,105.5860,105.4145,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
1,JNJ,85.8198,86.8303,86.8387,86.8893,87.0240,87.6724,87.0493,85.6514,84.6998,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
2,LLY,75.0504,75.1287,74.7981,74.4501,74.6763,75.2070,74.5719,72.5537,72.1187,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
3,PFE,24.0945,24.0650,24.0059,24.0798,24.0576,24.1832,23.9837,23.5258,23.7695,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
4,ABBV,42.0324,43.0286,43.4003,43.3185,43.8761,44.1586,44.2329,43.1624,43.1104,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837


Seems like some values are still missing... because the stock market be like that. Let's nuke them

In [28]:
big_boi = big_boi.dropna()
big_boi

,ticker,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-04,2016-01-05,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
0,UNH,106.7421,106.4802,107.0491,106.8053,107.3562,108.4038,107.1485,105.5860,105.4145,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
1,JNJ,85.8198,86.8303,86.8387,86.8893,87.0240,87.6724,87.0493,85.6514,84.6998,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
2,LLY,75.0504,75.1287,74.7981,74.4501,74.6763,75.2070,74.5719,72.5537,72.1187,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
3,PFE,24.0945,24.0650,24.0059,24.0798,24.0576,24.1832,23.9837,23.5258,23.7695,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
4,ABBV,42.0324,43.0286,43.4003,43.3185,43.8761,44.1586,44.2329,43.1624,43.1104,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,JAZZ,139.9900,144.1500,144.2400,141.7800,142.6500,142.5600,142.5500,137.8300,138.3500,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900
82,SRPT,37.9200,37.4800,38.3500,38.3100,37.8100,39.5000,38.5500,37.8500,38.2000,...,135.7900,130.6800,126.1900,126.3500,136.9900,135.7100,133.3900,133.0700,129.4500,127.6200
83,NVCR,27.5000,27.1200,26.3300,25.9100,24.9600,24.0100,22.8700,22.0000,22.7900,...,82.2500,82.6500,81.8200,85.3500,87.3400,88.7500,89.6400,87.5500,87.4700,84.9300
84,RDY,43.0524,43.2615,42.8908,44.3259,44.0123,44.3544,44.1548,43.4705,43.5466,...,39.2188,39.8740,39.7567,39.6002,40.0990,39.8251,39.6980,39.9719,39.7273,39.5904


Now all that's left is we need to remove the explicit date headings and instead label the dates as relative to the first date. 

In [29]:
num_days = len(big_boi.iloc[0]) - 1
column_names = ['ticker']
for i in range(num_days):
    day_id = f'day_{i}'
    column_names.append(day_id)
column_names

['ticker',
 'day_0',
 'day_1',
 'day_2',
 'day_3',
 'day_4',
 'day_5',
 'day_6',
 'day_7',
 'day_8',
 'day_9',
 'day_10',
 'day_11',
 'day_12',
 'day_13',
 'day_14',
 'day_15',
 'day_16',
 'day_17',
 'day_18',
 'day_19',
 'day_20',
 'day_21',
 'day_22',
 'day_23',
 'day_24',
 'day_25',
 'day_26',
 'day_27',
 'day_28',
 'day_29',
 'day_30',
 'day_31',
 'day_32',
 'day_33',
 'day_34',
 'day_35',
 'day_36',
 'day_37',
 'day_38',
 'day_39',
 'day_40',
 'day_41',
 'day_42',
 'day_43',
 'day_44',
 'day_45',
 'day_46',
 'day_47',
 'day_48',
 'day_49',
 'day_50',
 'day_51',
 'day_52',
 'day_53',
 'day_54',
 'day_55',
 'day_56',
 'day_57',
 'day_58',
 'day_59',
 'day_60',
 'day_61',
 'day_62',
 'day_63',
 'day_64',
 'day_65',
 'day_66',
 'day_67',
 'day_68',
 'day_69',
 'day_70',
 'day_71',
 'day_72',
 'day_73',
 'day_74',
 'day_75',
 'day_76',
 'day_77',
 'day_78',
 'day_79',
 'day_80',
 'day_81',
 'day_82',
 'day_83',
 'day_84',
 'day_85',
 'day_86',
 'day_87',
 'day_88',
 'day_89',
 'day_90'

In [30]:
big_boi.columns = column_names
big_boi

,ticker,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,...,day_1003,day_1004,day_1005,day_1006,day_1007,day_1008,day_1009,day_1010,day_1011,day_1012
0,UNH,106.7421,106.4802,107.0491,106.8053,107.3562,108.4038,107.1485,105.5860,105.4145,...,279.8966,279.6659,283.5101,287.2582,282.0013,283.7600,283.7984,284.4808,284.5192,282.2223
1,JNJ,85.8198,86.8303,86.8387,86.8893,87.0240,87.6724,87.0493,85.6514,84.6998,...,133.1297,133.8000,133.9583,136.7419,135.6806,135.9226,135.8947,136.1088,135.7178,135.0848
2,LLY,75.0504,75.1287,74.7981,74.4501,74.6763,75.2070,74.5719,72.5537,72.1187,...,118.4571,122.1636,123.3100,126.5675,126.4720,125.2110,125.1442,125.6887,125.6409,125.3161
3,PFE,24.0945,24.0650,24.0059,24.0798,24.0576,24.1832,23.9837,23.5258,23.7695,...,33.7570,33.3889,33.3804,33.6799,33.7398,33.6628,33.6371,33.7484,33.6371,33.2006
4,ABBV,42.0324,43.0286,43.4003,43.3185,43.8761,44.1586,44.2329,43.1624,43.1104,...,79.1581,79.2021,78.8766,79.5099,78.7447,79.5099,79.3340,79.1405,78.4721,77.5837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,JAZZ,139.9900,144.1500,144.2400,141.7800,142.6500,142.5600,142.5500,137.8300,138.3500,...,149.2200,149.2300,149.6200,153.0000,153.9500,153.5700,153.6200,152.4500,151.3200,148.0900
82,SRPT,37.9200,37.4800,38.3500,38.3100,37.8100,39.5000,38.5500,37.8500,38.2000,...,135.7900,130.6800,126.1900,126.3500,136.9900,135.7100,133.3900,133.0700,129.4500,127.6200
83,NVCR,27.5000,27.1200,26.3300,25.9100,24.9600,24.0100,22.8700,22.0000,22.7900,...,82.2500,82.6500,81.8200,85.3500,87.3400,88.7500,89.6400,87.5500,87.4700,84.9300
84,RDY,43.0524,43.2615,42.8908,44.3259,44.0123,44.3544,44.1548,43.4705,43.5466,...,39.2188,39.8740,39.7567,39.6002,40.0990,39.8251,39.6980,39.9719,39.7273,39.5904


In [34]:
big_boi.to_csv('new_stocks.csv')